# NOTE
This notebook was created for the sole purpose of generating maps for the project website; the code was run locally and might be slightly deprecated due to small changes between the datasets used here and the finalized versions linked to from Google Drive

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

In [2]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
#Download from: https://drive.google.com/open?id=1AOv3CrNOysm8FOfgSLL8aWhcWGPRXgPb
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [4]:
#Download from: https://drive.google.com/open?id=1-AtGVar_xNunGONvi1ANozOna3ZTriU8
listings = pd.read_csv('/home/novais/Downloads/listings.csv')

In [5]:
#Download from: https://drive.google.com/open?id=1-0f1ZkqJtLJkzlf7julTqM_IBHFBJtH8
crimes = pd.read_csv('/home/novais/Downloads/crimes.csv')

In [6]:
crimes.head()

,Unnamed: 0,date,latitude,longitude,arrest,desc,lat,lon,location,loc,loc_id,month,day,year
0,0,2018-07-09,41.894328,-87.628143,1,BATTERY,41.89,-87.63,"(41.894327845999996, -87.62814321)","(41.89, -87.63)",0,7,9,2018
1,462,2018-07-09,41.906323,-87.767016,0,BATTERY,41.91,-87.77,"(41.906323375, -87.76701638799999)","(41.91, -87.77)",6280,7,9,2018
2,463,2018-07-09,41.762420,-87.683181,0,BATTERY,41.76,-87.68,"(41.762419957, -87.68318080799999)","(41.76, -87.68)",6039,7,9,2018
3,464,2018-07-09,41.794497,-87.647608,1,CRIMINAL DAMAGE,41.79,-87.65,"(41.794496999, -87.647607611)","(41.79, -87.65)",6105,7,9,2018
4,465,2018-07-09,41.903997,-87.643230,1,CRIMINAL DAMAGE,41.90,-87.64,"(41.903996883000005, -87.64323023799999)","(41.9, -87.64)",6274,7,9,2018


In [7]:
crimes['Description']='Others'
crimes.loc[crimes.desc=="HOMICIDE",'Description']='Homicide'
crimes.loc[crimes.desc.isin(["BURGLARY", "THEFT", "ROBBERY", "MOTOR VEHICLE THEFT"]),'Description']="Stealing"
crimes.loc[crimes.desc.isin(["BATTERY", "ASSAULT"]),'Description']="Aggression"

In [8]:
crimes.Description.unique()

array(['Aggression', 'Others', 'Stealing', 'Homicide'], dtype=object)

In [9]:
colormap = {'Homicide': 'firebrick'}
crimes['location'] = crimes['location'].apply(lambda x: literal_eval(x))
crimes['coords_x'] = crimes['location'].apply(lambda x: merc(x)[0])
crimes['coords_y'] = crimes['location'].apply(lambda x: merc(x)[1])

crimesHomicide = crimes[crimes.Description =='Homicide']

ch0 = crimesHomicide[crimesHomicide['arrest']==0]
ch1 = crimesHomicide[crimesHomicide['arrest']==1]

p1 = figure(title = "Homicides in Chicago")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

source1 = ColumnDataSource(data=dict(
    Date=ch1['date'],
    cx=ch1['coords_x'],
    cy=ch1['coords_y'],
    colors = ['darkorange']*len(ch1),
    names = ['Arrested']*len(ch1)))
source0 = ColumnDataSource(data=dict(
    Date=ch0['date'],
    cx=ch0['coords_x'],
    cy=ch0['coords_y'],
    names = ['Not Arrested']*len(ch0),
    colors = ['firebrick']*len(ch0)))


p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',source=source0,size=10,color='colors',legend='names',line_color="darkred",line_width=1)
p1.circle(x = 'cx',y = 'cy',source=source1,size=10,color='colors',legend='names',line_color="tomato",line_width=1)

p1.add_tools(HoverTool(
    tooltips=[
        ( 'Date', '@Date{%F}')
    ],
    formatters={
        'Date' : 'datetime',   
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [10]:
output_notebook()


Loading BokehJS ...

In [12]:
show(p1)

In [13]:
print(p1_script)


<script type="text/javascript">
  (function() {
    var fn = function() {
      Bokeh.safely(function() {
        (function(root) {
          function embed_document(root) {
            
          var docs_json = '{"df6b5d03-d61d-407e-940c-8bd34d6b8a71":{"roots":{"references":[{"attributes":{},"id":"1125","type":"Selection"},{"attributes":{"formatter":{"id":"1053","type":"MercatorTickFormatter"},"plot":{"id":"1041","subtype":"Figure","type":"Plot"},"ticker":{"id":"1051","type":"MercatorTicker"}},"id":"1050","type":"MercatorAxis"},{"attributes":{"bottom_units":"screen","fill_alpha":{"value":0.5},"fill_color":{"value":"lightgrey"},"left_units":"screen","level":"overlay","line_alpha":{"value":1.0},"line_color":{"value":"black"},"line_dash":[4,4],"line_width":{"value":2},"plot":null,"render_mode":"css","right_units":"screen","top_units":"screen"},"id":"1076","type":"BoxAnnotation"},{"attributes":{},"id":"1046","type":"LinearScale"},{"attributes":{"dimension":"lon"},"id":"1051","type":"Mer

In [14]:
print(p1_div)


<div class="bk-root" id="c85a7b47-1a12-440e-9ed6-189fd9230ff5"></div>
